# Restricted Zone Notifier

This application uses the Inference Engine included in the Intel® Distribution of OpenVINO™ toolkit and the Intel® Deep Learning Deployment Toolkit. A trained neural network detects people within a marked assembly area, which is designed for a machine mounted camera system. It sends an alert if there is at least one person detected in the marked assembly area. The user can select the area coordinates via command line parameters. By default the whole frame is selected.

This sample is intended to demonstrate how to use Inference Engine included in the Intel® Distribution of OpenVINO™ toolkit and the Intel® Deep Learning Deployment Toolkit to improve assembly line safety for human operators and factory workers.

## Overview of how it works
At start-up the sample application reads the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. A job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and Intel® Arria® 10 FPGA.
After the inference is completed, the output videos are appropriately stored in the /results/[device] directory, which can then be viewed within the Jupyter Notebook instance.

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies which needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

### 0.2  (Optional-step): Original video without inference

To see the input video, run the following cell to view the original video stream used for inference and Restricted Zone.

In [ ]:
!ln -sf ./resources/worker-zone-detection.mp4 
videoHTML('Restricted Zone Video', ['worker-zone-detection.mp4'])

In [ ]:
rm -rf build && mkdir build

In [ ]:
cd build

### Compiling

In [ ]:
!cmake ..

In [ ]:
!make

In [ ]:
%%bash
./monitor -help

## Step 1: Using Intel® Distribution of OpenVINO™ toolkit

We will be using Intel® Distribution of OpenVINO™ toolkit Inference Engine (IE) to locate person in the frame.
There are five steps involved in this task:

1. Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel
2. Choose a device and create IEPlugin for the device
3. Read the IRModel using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### 1.1 Creating IR Model

The Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer. 
The Intel® Distribution of OpenVINO™ toolkit includes a utility script `model_downloader.py` that you can use to download some common models. Run the following cell to see the models available through `model_downloader.py`

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

**Note**: The '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a command line. So the above command will work straight out of the box on in a terminal (with '!' removed).

Some of these downloaded models are already in the IR format, while others will require the model optimizer. In this demo, we will be using the **person-detection-retail-0013** model, which is already in IR format. This model can be downloaded with the following command.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --help

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name pedestrian-detection-adas-0002 -o models

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


With the `-o` option set as above, this command downloads the model in the directory `models`, with the model files (.xml and .bin) located at `/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt`


## Step 2: Running the inference

### 2.1 Creating job file

To run inference on the video, we need more compute power.
We will run the workload on several edge compute nodes represented in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is written in Bash, and will be executed directly on the edge compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "restricted_zone_job.sh"

In [ ]:
%%writefile restricted_zone.sh

INPUT_FILE=$1
OUTPUT_FILE=$2
FP_MODEL=$3
BACKEND=$4
#0: CPU target (by default), 1: OpenCL, 2: OpenCL fp16 (half-float precision), 3: VPU,5: HETERO:FPGA,CPU
TARGET=$5


if [ "$TARGET" == "5" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/intel/init_openvino.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg1_bitstreams/2019R3_PV_PL1_FP16_MobileNet_Clamp.aocx
fi

cd $PBS_O_WORKDIR
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/

 
if [ "$FP_MODEL" == "FP16" ]; then
  FPEXT='-fp16'
fi

    
./monitor \
-m=models/intel/pedestrian-detection-adas-0002/$FP_MODEL/pedestrian-detection-adas-0002.bin \
-c=models/intel/pedestrian-detection-adas-0002/$FP_MODEL/pedestrian-detection-adas-0002.xml \
-i=$INPUT_FILE \
-o=$OUTPUT_FILE \
-b=$BACKENED \
-t=$TARGET

In [ ]:
!pbsnodes | grep compnode | sort | uniq -c

In [ ]:
os.environ["VIDEO"] = "../resources/worker-zone-detection.mp4"

### Submitting to an edge compute node with an Intel® Core™ CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel 
    Core i5-6500TE</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Core CPU...")
#Submit job to the queue
job_id_core = !qsub restricted_zone.sh -l nodes=1:tank-870:i5-6500te -F "$VIDEO ../results/ FP32 2 0"
print(job_id_core[0])

#Progress indicators
if job_id_core:
    progressIndicator('../results/', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
print("Submitting job to Intel Xeon CPU...")
#Submit job to the queue
job_id_xeon = !qsub restricted_zone.sh -l nodes=1:idc007xv5:e3-1268l-v5 -F "$VIDEO ../results/ FP32 2 0"
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('../results/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® Core™ CPU and using the onboard Intel GPU

In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub restricted_zone.sh -l nodes=1:idc001skl:intel-hd-530 -F "$VIDEO ../results/ FP32 2 1" 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('../results/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with Intel® Movidius™ Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
print("Submitting job to node with Intel NCS2...")
#Submit job to the queue
job_id_ncs2 = !qsub restricted_zone.sh -l nodes=1:idc004nc2:intel-ncs2 -F "$VIDEO ../results/ FP16 2 3"
print(job_id_ncs2[0])

#Progress indicators
if job_id_ncs2:
    progressIndicator('../results/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

### Submitting to an edge compute node with UP Squared Grove IoT Development Kit (UP2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub restricted_zone.sh -l nodes=1:up-squared -F "$VIDEO ../results/ FP16 2 1"
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('../results/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete.

***Wait!***

Please wait for the inference jobs and video rendering complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`obj_det_{type}.o{JobID}`

`obj_det_{type}.e{JobID}`

(here, obj_det_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/` directory.
We wrote a short utility script that will display these videos with in the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook.

In [ ]:
videoHTML('IEI Tank Core (Intel Core CPU)',
          ['results/output_'+job_id_core[0]+'.mp4'],
          '../results/stats_'+job_id_core[0]+'.txt')

In [ ]:
videoHTML('IEI Tank Core (Intel Xeon CPU)',
          ['results/output_'+job_id_xeon[0]+'.mp4'],
          '../results/stats_'+job_id_xeon[0]+'.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/output_'+job_id_gpu[0]+'.mp4'],
          '../results/stats_'+job_id_gpu[0]+'.txt')

In [ ]:
videoHTML('Tank-870:i5-6500te (Intel® NCS2)',
          ['results/output_'+job_id_ncs2[0]+'.mp4'],
          '../results/stats_'+job_id_ncs2[0]+'.txt')

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)',
          ['results/output_'+job_id_up2[0]+'.mp4'],
          '../results/stats_'+job_id_up2[0]+'.txt')

## Step 4: Assess Performance

The running time of each inference task is recorded in `results/*/stats.txt`, where the subdirectory name corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\ne3-1268l-v5'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('ncs2', 'Intel\nNCS2'),
             ('up2', 'Intel Atom\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        #stats_list.append(('../results/{arch}/stats.txt'.format(arch=arch), a_name))
        stats_list.append(('../results/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
       
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )